In [ ]:
# Tải dữ liệu từ Google Drive
!gdown --id 1Xn-aobJLKAq9APYOIENj4kYpk7Ox0NQH -O house-prices.zip
!unzip -o house-prices.zip

In [ ]:
# Tải kết quả tập test
!gdown --id 12LFeaeW9QJpEJPUAGxEipEh7h9rY66HU

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_log_error

In [ ]:
# Thiết lập matplotlib
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
np.random.seed(1)

In [ ]:
# Đọc dữ liệu
data_df = pd.read_csv('train.csv', index_col=0)
test_X = pd.read_csv('test.csv', index_col=0)
test_y = pd.read_csv('test_result.csv', index_col=0)['SalePrice']

In [ ]:
# Ý nghĩa của các cột
with open('data_description.txt', 'r') as f:
    print(f.read())

In [ ]:
# Các kiểu dữ liệu có trong bảng
data_df.dtypes.unique().tolist()

In [ ]:
# Thống kê sơ bộ cho các cột
cat_cols = data_df.dtypes.index[ data_df.dtypes == 'O' ].tolist()
num_cols = data_df.dtypes.index[ data_df.dtypes != 'O' ].tolist()
num_cols.remove('SalePrice')

In [ ]:
# Thống kê cho các cột có kiểu numeric
data_df[num_cols].describe()

In [ ]:
# Thống kê cho các cột có kiểu categorical
data_df[cat_cols].describe()

In [ ]:
# Mô tả dữ liệu
print("\nMô tả dữ liệu:")
print(data_df.describe())
print("\nThông tin dữ liệu:")
print(data_df.info())

In [ ]:
# Biểu đồ phân phối giá nhà trước và sau log-transform
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
sns.histplot(data_df['SalePrice'], bins=50, kde=True, ax=axes[0])
axes[0].set_title("Phân phối SalePrice ban đầu")
data_df["SalePrice"] = np.log1p(data_df["SalePrice"])
sns.histplot(data_df['SalePrice'], bins=50, kde=True, ax=axes[1])
axes[1].set_title("Phân phối SalePrice sau log-transform")
plt.show()

In [ ]:
# Xác định cột số và cột phân loại
cat_cols = data_df.select_dtypes(include=['object']).columns.tolist()
num_cols = data_df.select_dtypes(exclude=['object']).columns.tolist()
num_cols.remove('SalePrice')

In [ ]:
# Tiền xử lý dữ liệu
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='MissingValue')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, num_cols),
        ('categorical', categorical_transformer, cat_cols)
    ])

In [ ]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
data_X = data_df.drop('SalePrice', axis=1)
data_y = data_df['SalePrice']
train_X, val_X, train_y, val_y = train_test_split(data_X, data_y, test_size=0.3)

In [ ]:
# Hàm tính RMSLE
def root_MSLE(y_true, y_pred):
    return mean_squared_log_error(y_true, y_pred) ** (1/2)

In [ ]:
# Hàm đánh giá mô hình
def TestModel(model, name):
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', model)])
    pipeline.fit(train_X, train_y)
    val_pred = pipeline.predict(val_X)
    rmsle = root_MSLE(val_y, val_pred)
    print(f'RMSLE của {name}: {rmsle}')
    return pipeline, rmsle

In [ ]:
# Khởi tạo mô hình và chạy đánh giá
models = {
    "Gradient Boosting Regressor": GradientBoostingRegressor(random_state=22),
    "Ridge Regression": Ridge(random_state=22)
}

trained_models = {}
rmsle_scores = {}

for name, model in models.items():
    trained_models[name], rmsle_scores[name] = TestModel(model, name)


In [ ]:
# Hiển thị bảng so sánh RMSLE
rmsle_df = pd.DataFrame(list(rmsle_scores.items()), columns=["Model", "RMSLE"])
print("\nBảng so sánh RMSLE:")
print(rmsle_df)

In [ ]:
# So sánh kết quả bằng biểu đồ
plt.figure(figsize=(8, 5))
plt.bar(rmsle_scores.keys(), rmsle_scores.values(), color=['blue', 'green'])
plt.ylabel("RMSLE")
plt.title("So sánh RMSLE giữa các mô hình")
plt.show()

In [ ]:
# Dự đoán trên tập test (sử dụng pipeline để đảm bảo xử lý đúng)
final_gbr_pred = np.expm1(trained_models["Gradient Boosting Regressor"].predict(test_X))
final_ridge_pred = np.expm1(trained_models["Ridge Regression"].predict(test_X))

In [ ]:
# Lưu kết quả
gbr_results = pd.DataFrame({'Id': test_X.index, 'SalePrice': final_gbr_pred})
gbr_results.to_csv('gbr_final.csv', index=False)

ridge_results = pd.DataFrame({'Id': test_X.index, 'SalePrice': final_ridge_pred})
ridge_results.to_csv('ridge_final.csv', index=False)

In [ ]:
print("Dự đoán từ Gradient Boosting Regressor:")
print(final_gbr_pred[:10])  # Xem 10 dòng đầu

print("\nDự đoán từ Ridge Regression:")
print(final_ridge_pred[:10])

In [ ]:
results_df = pd.DataFrame({
    'Id': test_X.index,
    'GBR Prediction': final_gbr_pred,
    'Ridge Prediction': final_ridge_pred,
    'Actual Price': test_y.values  # Giá thực tế để so sánh
})
print(results_df.head(10))  # Xem 10 dòng đầu

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(test_y, final_gbr_pred, alpha=0.5, label="Gradient Boosting")
plt.scatter(test_y, final_ridge_pred, alpha=0.5, label="Ridge Regression")
plt.plot([min(test_y), max(test_y)], [min(test_y), max(test_y)], 'r--', label="Perfect Prediction")
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.legend()
plt.title("So sánh dự đoán và giá thực tế")
plt.show()

In [ ]:
# Hiển thị dữ liệu
gbr_results = pd.read_csv("gbr_final.csv")
ridge_results = pd.read_csv("ridge_final.csv")

print(gbr_results.head())  # Hiển thị 5 dòng đầu tiên
print(ridge_results.head())  # Hiển thị 5 dòng đầu tiên